In [15]:
from collections import defaultdict
from collections import deque
import sys

Class Node -> to store all values that corresponds to the node/id.
All necessary variables are declared below:

In [16]:
class Node:
    def __init__(self,id,name,dur,es,ef,ls,lf) -> None:
        self.name = name
        self.id = id ; self.du = dur
        self.es = es; self.ef = ef
        self.ls = ls; self.lf = lf
    
nodes = {}
visited = {}
q = deque()
forwardPass = defaultdict(list)
backwardPass = defaultdict(list)
criticalPath = []
totalDuration = 0
leaves = {} 

Reading input file and Storing all the corresponding value.
nodes dictionary used to store the object of Node class.
All the id appended to the queue which have no predecessors.

In [17]:
path = './input3.txt'
for lines in open(path):
    line = lines.rstrip('\n').split(',')
    id = int(line[0]); name = line[1]; duration = int(line[2])
    ef = 0
    if len(line) == 4:
        predecessors = line[3].split(';')
        for x in predecessors:
            x = int(x)
            forwardPass[x].append(id)
            backwardPass[id].append(x)
    else:
        q.append(id)
        ef = duration

    nodes[id] = Node(id,name,duration,0,ef,0,sys.maxsize)
    visited[id] = 0
    leaves[id] = 0

ForwardPass:
    Traverse through the forwardPass using BFS.
    Update es and ef --- es is updated with maximum value of its predecessors.
    outDegree is used to find the leaves from which we'll start backwardPass.

In [18]:
while q:
    outDegree = 0
    u = q.popleft()
    ef = nodes[u].ef
    for v in forwardPass[u]:
        if visited[v] != 1:
            nodes[v].es = max(ef,nodes[v].es)
            nodes[v].ef = nodes[v].es + nodes[v].du
            q.append(v)
            outDegree += 1

    if outDegree == 0 and visited[u] == 0:
        leaves[u] = 1
        totalDuration = max(totalDuration,nodes[u].ef)
        
    visited[u] = 1

BackwardPass:
    All leaf nodes are added to the queue and corresponding ls , lf is calculated.
    Using BFS forwardPass is traversed in backward direction.
    lf is calculated as minimum(ls) of its successors.

In [19]:
for node,leaf in leaves.items():
    if leaf:
        nodes[node].lf = totalDuration
        nodes[node].ls = nodes[node].lf - nodes[node].du
        q.append(node)
    visited[node] = 0

while q:
    u = q.popleft()
    ls = nodes[u].ls
    for v in backwardPass[u]:
        if visited[v] != 1:
            nodes[v].lf = min(nodes[v].lf,ls)
            nodes[v].ls = nodes[v].lf - nodes[v].du
            q.append(v)
    
    if nodes[u].es == nodes[u].ls and visited[u] == 0:
        criticalPath.append(nodes[u].name)
        
    visited[u] = 1
    

In [20]:
def printNodes(nodes):
    for key,value in nodes.items():
        print('id = {},node = {},duration = {},ES = {},EF = {},LS = {},LF = {}'
            .format(key,value.name,value.du,value.es,value.ef,value.ls,value.lf))
            
printNodes(nodes)

id = 1,node = A,duration = 12,ES = 0,EF = 12,LS = 2,LF = 14
id = 2,node = B,duration = 6,ES = 23,EF = 29,LS = 23,LF = 29
id = 3,node = E,duration = 12,ES = 12,EF = 24,LS = 20,LF = 32
id = 4,node = F,duration = 18,ES = 12,EF = 30,LS = 14,LF = 32
id = 5,node = C,duration = 5,ES = 29,EF = 34,LS = 29,LF = 34
id = 6,node = G,duration = 10,ES = 30,EF = 40,LS = 32,LF = 42
id = 7,node = I,duration = 8,ES = 30,EF = 38,LS = 34,LF = 42
id = 8,node = D,duration = 8,ES = 34,EF = 42,LS = 34,LF = 42
id = 100,node = X,duration = 23,ES = 0,EF = 23,LS = 0,LF = 23


In [21]:
for i , e  in reversed(list(enumerate(criticalPath))):
    print(e,end = '')
    if i != 0:
        print('->',end='')

X->B->C->D